## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%capture
!wget "https://www.dropbox.com/s/5sp14xcueavt6dc/women-clothing-accessories.csv"

data = pd.read_csv("women-clothing-accessories.csv", sep='\t', usecols=[0, 1])

In [ ]:
data

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     90000 non-null  object
 1   sentiment  90000 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


Нулевые значения отсутствуют - данные полные.

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 7.9 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=a9dee4de3351bbb6f662c6a000a8ae3b099d08e87aeb3992dc992b6de81228f5
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
pymorphy2_analyzer = MorphAnalyzer()
noise = stopwords.words('russian')

data_copy = data.copy()
for i in range(data_copy.shape[0]):
  sentense = data_copy['review'][i]
  sentense = sentense.lower()
  words = sentense.split()
  # as we use russian lang
  sent = ""
  for j in range(len(words)):
    words[j] = pymorphy2_analyzer.parse(words[j])[0].normal_form
    sent += ' ' + words[j]
  data_copy['review'][i] = sent

In [ ]:
data_copy

,review,sentiment
0,качество плохой пошив ужасный (горловина напе...,negative
1,"товар отдать другой человеку, я не получить п...",negative
2,"ужасный синтетика! тонкая, ничего общий с пре...",negative
3,"товар не пришел, продавец продлить защита без...",negative
4,"кофточка голый синтетика, носить не возможно.",negative
...,...,...
89995,сделать достаточно хорошо. на ткань сделать р...,positive
89996,накидка шикарная. спасибо большой провдо линя...,positive
89997,спасибо большой ) продовца рекомендую.. заказ...,positive
89998,очень довольный заказом! маленький месяц в рб...,positive


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_copy.review, data_copy.sentiment, train_size = 0.7)

**Для векторайзора: мешок n-грамм**

In [ ]:
for i in range(1, 7):
  for j in range(i + 1, 7):
    vectorizor = CountVectorizer(analyzer='word', ngram_range=(i,j),lowercase=True,stop_words=noise, tokenizer=nltk.wordpunct_tokenize)
    vectorize_x_train = vectorizor.fit_transform(x_train)

    clf1 = MultinomialNB()
    clf1.fit(vectorize_x_train, y_train)

    vectorize_x_test = vectorizor.transform(x_test)
    y_pred = clf1.predict(vectorize_x_test)

    print("Мешок n-грамм: ", i, j)
    print(classification_report(y_test, y_pred))

Мешок n-грамм:  1 2
              precision    recall  f1-score   support

    neautral       0.61      0.63      0.62      9127
    negative       0.72      0.64      0.68      8978
    positive       0.81      0.87      0.84      8896

    accuracy                           0.71     27001
   macro avg       0.71      0.71      0.71     27001
weighted avg       0.71      0.71      0.71     27001

Мешок n-грамм:  1 3
              precision    recall  f1-score   support

    neautral       0.62      0.60      0.61      9127
    negative       0.72      0.66      0.69      8978
    positive       0.79      0.88      0.83      8896

    accuracy                           0.71     27001
   macro avg       0.71      0.71      0.71     27001
weighted avg       0.71      0.71      0.71     27001

Мешок n-грамм:  1 4
              precision    recall  f1-score   support

    neautral       0.62      0.58      0.60      9127
    negative       0.71      0.67      0.69      8978
    positive   

Проверив построеную модель обучения, где метрики подбирались через Мешок n-грамм, с разным значением диапазона n, видно, что наулучшее значение = (1,2).

**TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import re

words_tab = []
for i in range(data_copy.shape[0]):
  sent = data_copy['review'][i]
  sent = re.sub("[^а-яА-ЯёЁa-zA-Z]"," ", sent)
  words = sent.split()
  words = [w for w in words if not w in noise]

  for j in range(len(words)):
    words_tab.append(words[j])

words_dic = pd.DataFrame(words_tab, columns=list('w'))
words_dic

,w
0,качество
1,плохой
2,пошив
3,ужасный
4,горловина
...,...
1228572,хххl
1228573,село
1228574,идеально
1228575,лёгкая


In [ ]:
words_dic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228577 entries, 0 to 1228576
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   w       1228577 non-null  object
dtypes: object(1)
memory usage: 9.4+ MB


In [ ]:
print(len(words_tab))

1228577


In [ ]:
words_dic.groupby('w').size().sort_values(ascending=False)

w
очень       29236
размер      25900
продавец    19698
качество    15134
товар       14992
            ...  
деньой          1
денюжки         1
денюжку         1
пересыла        1
ёё              1
Length: 42346, dtype: int64

In [ ]:
print(29236.0/len(words_tab))

0.023796636271068074


In [ ]:
for i in range(1, 7):
  for j in range(i + 1, 7):
    vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(i,j),
                                  lowercase=True,stop_words=noise,
                                  tokenizer=nltk.wordpunct_tokenize)
    vectorize_x_train = vectorizor.fit_transform(x_train)

    clf1 = MultinomialNB()
    clf1.fit(vectorize_x_train, y_train)

    vectorize_x_test = vectorizor.transform(x_test)
    y_pred = clf1.predict(vectorize_x_test)

    print("TF-IDF: ", i, j, "max_df= ", 1.0, "min_df= ", 0.0)
    print(classification_report(y_test, y_pred, digits=4))

TF-IDF:  1 2 max_df=  1.0 min_df=  0.0
              precision    recall  f1-score   support

    neautral     0.6162    0.6051    0.6106      9127
    negative     0.7265    0.6478    0.6849      8978
    positive     0.7850    0.8852    0.8321      8896

    accuracy                         0.7116     27001
   macro avg     0.7092    0.7127    0.7092     27001
weighted avg     0.7085    0.7116    0.7083     27001

TF-IDF:  1 3 max_df=  1.0 min_df=  0.0
              precision    recall  f1-score   support

    neautral     0.6158    0.5793    0.5970      9127
    negative     0.7261    0.6527    0.6875      8978
    positive     0.7651    0.8897    0.8227      8896

    accuracy                         0.7060     27001
   macro avg     0.7023    0.7072    0.7024     27001
weighted avg     0.7017    0.7060    0.7014     27001

TF-IDF:  1 4 max_df=  1.0 min_df=  0.0
              precision    recall  f1-score   support

    neautral     0.6161    0.5786    0.5968      9127
    negative

In [ ]:
max_dfs = np.arange(0.012, 0.024, 0.002)
min_dfs = np.arange(0.00001, 0.00011, 0.00001)

for max_df in max_dfs:
  for min_df in min_dfs:
    vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1,2),
                                  lowercase=True,stop_words=noise,
                                  tokenizer=nltk.wordpunct_tokenize,
                                  max_df=round(max_df, 3),
                                  min_df=round(min_df, 5))
    vectorize_x_train = vectorizor.fit_transform(x_train)

    clf1 = MultinomialNB()
    clf1.fit(vectorize_x_train, y_train)

    vectorize_x_test = vectorizor.transform(x_test)
    y_pred = clf1.predict(vectorize_x_test)

    print("TF-IDF: ", 1, 2, "max_df= ", round(max_df, 3), "min_df= ", round(min_df, 5))
    print(classification_report(y_test, y_pred, digits=4))

TF-IDF:  1 2 max_df=  0.012 min_df=  1e-05
              precision    recall  f1-score   support

    neautral     0.6139    0.6018    0.6078      9127
    negative     0.7232    0.6327    0.6749      8978
    positive     0.7664    0.8787    0.8187      8896

    accuracy                         0.7033     27001
   macro avg     0.7012    0.7044    0.7005     27001
weighted avg     0.7005    0.7033    0.6996     27001

TF-IDF:  1 2 max_df=  0.012 min_df=  2e-05
              precision    recall  f1-score   support

    neautral     0.6070    0.6211    0.6140      9127
    negative     0.7233    0.6261    0.6712      8978
    positive     0.7807    0.8679    0.8220      8896

    accuracy                         0.7041     27001
   macro avg     0.7037    0.7050    0.7024     27001
weighted avg     0.7029    0.7041    0.7015     27001

TF-IDF:  1 2 max_df=  0.012 min_df=  3e-05
              precision    recall  f1-score   support

    neautral     0.6070    0.6211    0.6140      9127


Подобранные парамметры:
* n-range (1, 2)
* max_df = 0.022
* min_df = 0.00007

**Cимвольные n-граммы**

In [ ]:
for i in range(1, 7):
  for j in range(i + 1, 7):
    vectorizor = CountVectorizer(analyzer='char', ngram_range=(i,j),lowercase=True)
    vectorize_x_train = vectorizor.fit_transform(x_train)

    clf1 = MultinomialNB()
    clf1.fit(vectorize_x_train, y_train)

    vectorize_x_test = vectorizor.transform(x_test)
    y_pred = clf1.predict(vectorize_x_test)

    print("Символьные n-граммы: ", i, j)
    print(classification_report(y_test, y_pred, digits=4))

Символьные n-граммы:  1 2
              precision    recall  f1-score   support

    neautral     0.5321    0.6573    0.5881      9127
    negative     0.6781    0.5664    0.6172      8978
    positive     0.7704    0.7126    0.7404      8896

    accuracy                         0.6453     27001
   macro avg     0.6602    0.6454    0.6486     27001
weighted avg     0.6592    0.6453    0.6480     27001

Символьные n-граммы:  1 3
              precision    recall  f1-score   support

    neautral     0.5624    0.6908    0.6200      9127
    negative     0.7103    0.5867    0.6426      8978
    positive     0.8282    0.7797    0.8032      8896

    accuracy                         0.6855     27001
   macro avg     0.7003    0.6857    0.6886     27001
weighted avg     0.6991    0.6855    0.6879     27001

Символьные n-граммы:  1 4
              precision    recall  f1-score   support

    neautral     0.5780    0.7034    0.6346      9127
    negative     0.7225    0.5992    0.6551      89

n-range = (5,6)

In [ ]:
for i in range(6, 10):
  for j in range(i + 1, 10):
    vectorizor = CountVectorizer(analyzer='char', ngram_range=(i,j),lowercase=True)
    vectorize_x_train = vectorizor.fit_transform(x_train)

    clf1 = MultinomialNB()
    clf1.fit(vectorize_x_train, y_train)

    vectorize_x_test = vectorizor.transform(x_test)
    y_pred = clf1.predict(vectorize_x_test)

    print("Символьные n-граммы: ", i, j)
    print(classification_report(y_test, y_pred, digits=4))

Символьные n-граммы:  6 7
              precision    recall  f1-score   support

    neautral     0.6115    0.6994    0.6525      9127
    negative     0.7327    0.6410    0.6838      8978
    positive     0.8745    0.8561    0.8652      8896

    accuracy                         0.7316     27001
   macro avg     0.7396    0.7322    0.7338     27001
weighted avg     0.7385    0.7316    0.7330     27001

Символьные n-граммы:  6 8
              precision    recall  f1-score   support

    neautral     0.6153    0.6938    0.6522      9127
    negative     0.7308    0.6490    0.6875      8978
    positive     0.8752    0.8596    0.8674      8896

    accuracy                         0.7335     27001
   macro avg     0.7405    0.7341    0.7357     27001
weighted avg     0.7394    0.7335    0.7348     27001

Символьные n-граммы:  6 9
              precision    recall  f1-score   support

    neautral     0.6198    0.6887    0.6524      9127
    negative     0.7306    0.6568    0.6917      89

Увеличив значение диапазона, получаем более точные значения примерно до (6, 9), далее точность падает. А значит, лучшее значение для модели диапазон (6, 9).

**Сравнение:**

In [ ]:
clf = MultinomialNB()

# n-range
clf.fit(vectorize_x_train, y_train)
vectorizor = CountVectorizer(analyzer='word', ngram_range=(1,2),
                             lowercase=True,stop_words=noise,
                             tokenizer=nltk.wordpunct_tokenize)
vectorize_x_train = vectorizor.fit_transform(x_train)
clf.fit(vectorize_x_train, y_train)

vectorize_x_test = vectorizor.transform(x_test)

y_pred1 = clf.predict(vectorize_x_test)
report1 = classification_report(y_test, y_pred1, digits=4, output_dict=True)

# td-idf
clf.fit(vectorize_x_train, y_train)
vectorizor = TfidfVectorizer(analyzer='word', ngram_range=(1,2),
                                  lowercase=True,stop_words=noise,
                                  tokenizer=nltk.wordpunct_tokenize,
                                  max_df=round(0.022, 3),
                                  min_df=round(0.00007, 5))
vectorize_x_train = vectorizor.fit_transform(x_train)
clf.fit(vectorize_x_train, y_train)

vectorize_x_test = vectorizor.transform(x_test)

y_pred2 = clf.predict(vectorize_x_test)
report2 = classification_report(y_test, y_pred2, digits=4, output_dict=True)

# n-range char
clf.fit(vectorize_x_train, y_train)
vectorizor = CountVectorizer(analyzer='char', ngram_range=(6,9),lowercase=True)
vectorize_x_train = vectorizor.fit_transform(x_train)
clf.fit(vectorize_x_train, y_train)

vectorize_x_test = vectorizor.transform(x_test)

y_pred3 = clf.predict(vectorize_x_test)
report3 = classification_report(y_test, y_pred3, digits=4, output_dict=True)

In [ ]:
report_dict = [['Мешок n-грамм',
                report1['positive']['precision'],
                report1['positive']['recall'],
                report1['positive']['f1-score'],
                report1['negative']['precision'],
                report1['negative']['recall'],
                report1['negative']['f1-score'],
                report1['neautral']['precision'],
                report1['neautral']['recall'],
                report1['neautral']['f1-score'],
                report1['macro avg']['precision'],
                report1['macro avg']['recall'],
                report1['macro avg']['f1-score'],
                report1['weighted avg']['precision'],
                report1['weighted avg']['recall'],
                report1['weighted avg']['f1-score']],
               ['Tf-idf',
                report2['positive']['precision'],
                report2['positive']['recall'],
                report2['positive']['f1-score'],
                report2['negative']['precision'],
                report2['negative']['recall'],
                report2['negative']['f1-score'],
                report2['neautral']['precision'],
                report2['neautral']['recall'],
                report2['neautral']['f1-score'],
                report2['macro avg']['precision'],
                report2['macro avg']['recall'],
                report2['macro avg']['f1-score'],
                report2['weighted avg']['precision'],
                report2['weighted avg']['recall'],
                report2['weighted avg']['f1-score']],
               ['Символьные n-граммы',
                report3['positive']['precision'],
                report3['positive']['recall'],
                report3['positive']['f1-score'],
                report3['negative']['precision'],
                report3['negative']['recall'],
                report3['negative']['f1-score'],
                report3['neautral']['precision'],
                report3['neautral']['recall'],
                report3['neautral']['f1-score'],
                report3['macro avg']['precision'],
                report3['macro avg']['recall'],
                report3['macro avg']['f1-score'],
                report3['weighted avg']['precision'],
                report3['weighted avg']['recall'],
                report3['weighted avg']['f1-score']]]
report_table = pd.DataFrame(report_dict, columns=['Векторайзер', 'positive precision','positive recall','positive f1-score',
                                         'negative precision','negative recall','negative f1-score',
                                         'neautral precision','neautral recall','neautral f1-score',
                                         'macro avg precision','macro avg recall','macro avg f1-score',
                                         'weighted avg precision','weighted avg recall','weighted avg f1-score'])
report_table

,Векторайзер,positive precision,positive recall,positive f1-score,negative precision,negative recall,negative f1-score,neautral precision,neautral recall,neautral f1-score,macro avg precision,macro avg recall,macro avg f1-score,weighted avg precision,weighted avg recall,weighted avg f1-score
0,Мешок n-грамм,0.811949,0.869267,0.839631,0.720895,0.642125,0.679234,0.609177,0.632738,0.620734,0.714007,0.714710,0.713200,0.713131,0.713788,0.712305
1,Tf-idf,0.809783,0.854204,0.831400,0.729044,0.633549,0.677950,0.604687,0.650268,0.626650,0.714504,0.712674,0.712000,0.713609,0.711900,0.711166
2,Символьные n-граммы,0.873336,0.862635,0.867952,0.730550,0.656828,0.691730,0.619799,0.688726,0.652447,0.741228,0.736063,0.737376,0.740157,0.735417,0.736511


Судя по данным, лучше метод отрабатывает с символьным n-граммным векторайзером - значения для по всем столбцам с оценками для него выше.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

Потому что префикс 'a' подстроки уже был включен в abca. Findall ищет непересекающиеся шаблоны.

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
def res(str):
  find = re.findall(r'\b\b[а-яА-Яa-zA-Z][а-яА-Яa-zA-Z]', str)
  return find
print(res('adyuy asfd 1sdg'))

['ad', 'as']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
def res(str):
  find = re.split('[A-Z]*[a-zA-Z]\. ', str, maxsplit=3)
  return find
print(res('Adyuy. As.fd 1sdg. Gggggg, g g ggg. aaaAaaa.'))

['Adyu', 'As.fd 1sd', 'Gggggg, g g gg', 'aaaAaaa.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('\d', 'DIG', '12aaaa r34 aa')
print(result)

DIGDIGaaaa rDIGDIG aa


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('(http[s]?://|www\.)(?:[а-яА-Яa-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',
                'text https://787someurl/a a www.otherurl f')
print(result)

text  a  f


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile(r'[а-яА-Яa-zA-Z]{4,}')
prog.findall("aaa abcd abcddd as a qwe qwerty")

['abcd', 'abcddd', 'qwerty']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile(r'@[а-яА-Яa-zA-Z\d_]+\.[а-яА-Яa-zA-Z\d_\.]{1,4}\b')
prog.findall('abc.test@gmail.comggg, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@test.in', '@analyticsvidhya.com', '@rest.biz']